In [1]:
print("Hello WOrld")

Hello WOrld


In [2]:
import hashlib
from pathlib import Path

from langchain_qdrant import QdrantVectorStore, RetrievalMode
from langchain_core.documents import Document
from helpers.common import (
    client,
    vector_store,
    dense_embeddings,
    sparse_embeddings,
    COLLECTION_NAME_TOGETHER,
    URL,
)

/home/yash/Desktop/Code/KGP/rag_venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# client from helpers.common (used as qdrant_client in this notebook)
qdrant_client = client

In [4]:
# dense_embeddings from helpers.common


No sentence-transformers model found with name togethercomputer/m2-bert-80M-8k-retrieval. Creating a new one with mean pooling.
You are using a model of type m2_bert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
/home/yash/.cache/huggingface/modules/transformers_modules/togethercomputer/m2_hyphen_bert_hyphen_80M_hyphen_8k_hyphen_retrieval/90e0b28f3382c289cc6c1e92ef53d7dc5a3ec14b/blockdiag_multiply.py:50: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.bfloat16)
/home/yash/.cache/huggingface/modules/transformers_modules/togethercomputer/m2_hyphen_bert_hyphen_80M_hyphen_8k_hyphen_retrieval/90e0b28f3382c289cc6c1e92ef53d7dc5a3ec14b/blockdiag_multiply.py:63: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` in

-- Bidirectional: True
-- Using Long Conv Residual: True
-- Hyena w: 10
-- Hyena w mod: 1
-- Hyena filter order: 128
-- Hyena filter dropout: 0.2
-- Hyena filter wd: 0.1
-- Hyena filter emb dim: 5
-- Hyena filter lr: 0.001
-- Hyena filter lr pos emb: 1e-05


In [5]:
# sparse_embeddings, COLLECTION_NAME_TOGETHER from helpers.common


In [6]:
vector_store.client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='e45f6e7f-c0d5-486f-8a70-a62e1ba8e925'), CollectionDescription(name='134ec4c047a5436589e9ce778624f02f'), CollectionDescription(name='pavestone_old'), CollectionDescription(name='blueocean')])

In [9]:
# client from helpers.common
COLLECTION_NAME_ = "pavestone_old"

def extract_all_documents(collection_name):
    """Scroll through entire collection and extract points"""
    all_points = []
    offset = None
    
    while True:
        results, offset = client.scroll(
            collection_name=collection_name,
            limit=100,
            offset=offset,
            with_payload=True,
            with_vectors=False  # Don't need old vectors
        )
        all_points.extend(results)
        
        if offset is None:
            break
    
    print(f"Extracted {len(all_points)} points from '{collection_name}'")
    return all_points

In [10]:
all_points = extract_all_documents(COLLECTION_NAME_)

Extracted 167 points from 'pavestone_old'


In [8]:
def points_to_documents(points):
    docs = []
    for point in points:
        payload = point.payload
        
        # Your text is stored under "text" key
        text = payload.get("text", "")
        
        # All other fields are your metadata — stored flat
        metadata = {
            "organization_id": payload.get("organization_id"),
            "project_id": payload.get("project_id"),
            "notebook_id": payload.get("notebook_id"),
            "source_file_id": payload.get("source_file_id"),
            "file_name": payload.get("file_name"),
            "file_path": payload.get("file_path"),
            "page_start": payload.get("page_start"),
            "page_end": payload.get("page_end"),
            "chunk_index": payload.get("chunk_index"),
        }
        
        docs.append(Document(page_content=text, metadata=metadata))
    
    return docs

In [11]:
points_to_documents(all_points)

[Document(metadata={'organization_id': 'e5b04358-7fa0-4670-a562-771855d6b78c', 'project_id': '0621d3bd-84ff-49e0-a20f-c7b7d3d758a6', 'notebook_id': None, 'source_file_id': '657998ca-2bde-46ba-9517-907f87e5d28c', 'file_name': '01_23.01.26_Pavestone-Advarisk.pdf', 'file_path': 'uploads/source_files/pavestone/Advarisk/Shramay_Jha/657998ca-2bde-46ba-9517-907f87e5d28c_01_23.01.26_Pavestone-Advarisk.pdf', 'page_start': 12, 'page_end': 12, 'chunk_index': 11}, page_content="accessible and usable for the financial institution. So by standardizing the languages through transliteration \nengines, by identifying, because it's all text and unstructured data, different formats. \n  \n \n \n \n \nVishal - AdvaRisk - 10:09\n \n \n Right. \n  \n \n \n \n \nVishal - AdvaRisk - 10:10\n \n \n Different way of writing things. You say in Maharashtra, if there is a satisfaction of charge in a land record, they will \nsimply strike that line with says loan from so&so bank in Marathi but in say a madhya curren

In [12]:
OLD_COLLECTIONS = [COLLECTION_NAME_] 
NEW_COLLECTION_SUFFIX = "_v2" 
# c55255ea-de8b-4b6c-965a-d4f5557e7725
def migrate_collection(old_name, new_name):
    print(f"Migrating '{old_name}' → '{new_name}'")
    
    points = extract_all_documents(old_name)
    docs = points_to_documents(points)
    print(f"  Found {len(docs)} documents")
    
    if not docs:
        print("  Skipping — empty collection")
        return
    
    vector_store = QdrantVectorStore.from_documents(
        documents=docs,
        embedding=dense_embeddings,
        sparse_embedding=sparse_embeddings,
        url=URL,
        collection_name=new_name,
        retrieval_mode=RetrievalMode.HYBRID,
        force_recreate=True 
    )
    print("  Done ✓")
    return vector_store

# Run migration
for col in OLD_COLLECTIONS:
    migrate_collection(col, col + NEW_COLLECTION_SUFFIX)

Migrating 'pavestone_old' → 'pavestone_old_v2'
Extracted 167 points from 'pavestone_old'
  Found 167 documents
  Done ✓


In [21]:
path = Path("e45f6e7f-c0d5-486f-8a70-a62e1ba8e925-3893554715309385-2026-02-17-04-50-53.snapshot")

path

PosixPath('e45f6e7f-c0d5-486f-8a70-a62e1ba8e925-3893554715309385-2026-02-17-04-50-53.snapshot')

In [20]:
import requests

snapshot_path = path

with open(snapshot_path, "rb") as f:
    response = requests.post(
        "http://localhost:6333/collections/your_collection_name/snapshots/upload",
        files={"snapshot": f},
        params={"priority": "snapshot"}
    )

print(response.json())

{'result': True, 'status': 'ok', 'time': 0.405781626}


In [ ]:
PROD_ORG_ID = 'c55255ea-de8b-4b6c-965a-d4f5557e7725'
PAVESTONE_ORG_ID = 'e5b04358-7fa0-4670-a562-771855d6b78c'
BLUEOCEAN_ORG_ID='73ebc657-8378-4f95-9af5-a00c8ac11aa5'


In [ ]:
# curl http://sparlm.com:9000/collections/c55255ea-de8b-4b6c-965a-d4f5557e7725
